In [3]:
import re
import csv

In [4]:
filename = "airline.csv"

exceptionsList = ["CRJ", "ATR", "USD", "FLT", "IFE", "USD", "ERJ", "CRJ", "EMB", "ATR", "AND", "BUT", "FOR", "THE", "FLT", "EUR"]
regexStr = ""

allAirportCodes_pre=[]
allAirportCodes_post=[]

fields = []
rows = []

In [5]:
def readCSV():
  newRows = []
  global allAirportCodes_post
  with open(filename, 'r', encoding='utf-8') as csvfile:
      csvreader = csv.reader(csvfile)
      fields = next(csvreader)

      for row in csvreader:
          #index 6 is where str is contained
          airCodes = isAirport(row[6])
          newRows.append(airCodes)
  #sloppy code but works hopefully
  uniqueAir = set(allAirportCodes_pre)
  allAirportCodes_post = list(uniqueAir)
  writeToCSV(newRows)

In [6]:
def deconstructAirCode(acronymList):
  deconstructed = []
  global allAirportCodes_post
  for code in allAirportCodes_post:
      if code in acronymList:
        deconstructed.append("1")
      else:
        deconstructed.append("0")
  return deconstructed

In [7]:
def writeToCSV(newRows):
  global allAirportCodes_post
  with open('airlines_fixed.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(allAirportCodes_post)
    for row in newRows:
      #removes and returns the airport code list at end
      airCodeList = deconstructAirCode(row)
      writer.writerow(airCodeList)

In [8]:
def createRegEx():
  global regexStr
  for ex in exceptionsList: 
    toAdd = "(?!" + ex + ")"
    regexStr += toAdd
  regexStr += "([A-Z]{3})"
  return regexStr

In [9]:
def advancedFilter(tupleVar, str):
  start = tupleVar[0]
  end = tupleVar[1]
  charBefore = str[start - 1: start]
  charAfter = str[end : end + 1]
  #characters that are allowed to exist before and after the airport code
  possibleChars=[" ", "/", "-", ".", "", "(", ")", "[", "]", "\\"]
  if((charBefore in possibleChars) and (charAfter in possibleChars)):
    return True
  else:
    return False

In [10]:
def isAirport(str):
  global pattern
  global allAirportCodes_pre
  validList = []
  #creates a list of the start and end indices for each Airport code found in the list
  indexTuples = [(m.start(0), m.end(0)) for m in re.finditer(pattern, str)]
  for tupleVar in indexTuples:
    isValid = advancedFilter(tupleVar, str)
    if(isValid):
      subStr = str[tupleVar[0]:tupleVar[1]]
      validList.append(subStr)
      #sloppy and redundant but the following line works lol
      allAirportCodes_pre.append(subStr)
  #get only unique items list
  uniqueList = set(validList) 
  #convert set back to list
  finalRowUnique = list(uniqueList)
  return finalRowUnique

In [11]:
pattern = createRegEx()
readCSV()